In [1]:
import warnings
warnings.filterwarnings("ignore")

from xai_agg.xai_agg import *


from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import NearestNeighbors


2024-12-03 23:59:37.376301: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-03 23:59:37.384379: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733281177.394259  117317 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733281177.397200  117317 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 23:59:37.407553: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Preprocess the data

In [2]:
original_data = pd.read_csv('data/german_credit_data_updated.csv')

# Dataset overview - German Credit Risk (from Kaggle):
# 1. Age (numeric)
# 2. Sex (text: male, female)
# 3. Job (numeric: 0 - unskilled and non-resident, 1 - unskilled and resident, 2 - skilled, 3 - highly skilled)
# 4. Housing (text: own, rent, or free)
# 5. Saving accounts (text - little, moderate, quite rich, rich)
# 6. Checking account (numeric, in DM - Deutsch Mark)
# 7. Credit amount (numeric, in DM)
# 8. Duration (numeric, in month)
# 9. Purpose (text: car, furniture/equipment, radio/TV, domestic appliances, repairs, education, business, vacation/others)

display(original_data.head())
display(original_data.describe())
display(original_data.info())

# Display the unique values of the categorical features:
print('Unique values of the categorical features:')
for col in original_data.select_dtypes(include='object'):
    print(f'\t- {col}: {original_data[col].unique()}')

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Credit Risk
0,0,67,male,2,own,NaN,little,1169,6,radio/TV,1
1,1,22,female,2,own,little,moderate,5951,48,radio/TV,2
2,2,49,male,1,own,little,NaN,2096,12,education,1
3,3,45,male,2,free,little,little,7882,42,furniture/equipment,1
4,4,53,male,2,free,little,little,4870,24,car,2


,Unnamed: 0,Age,Job,Credit amount,Duration,Credit Risk
count,954.000000,954.000000,954.000000,954.000000,954.000000,954.000000
mean,476.500000,35.501048,1.909853,3279.112159,20.780922,1.302935
std,275.540378,11.379668,0.649681,2853.315158,12.046483,0.459768
min,0.000000,19.000000,0.000000,250.000000,4.000000,1.000000
25%,238.250000,27.000000,2.000000,1360.250000,12.000000,1.000000
50%,476.500000,33.000000,2.000000,2302.500000,18.000000,1.000000
75%,714.750000,42.000000,2.000000,3975.250000,24.000000,2.000000
max,953.000000,75.000000,3.000000,18424.000000,72.000000,2.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954 entries, 0 to 953
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        954 non-null    int64 
 1   Age               954 non-null    int64 
 2   Sex               954 non-null    object
 3   Job               954 non-null    int64 
 4   Housing           954 non-null    object
 5   Saving accounts   779 non-null    object
 6   Checking account  576 non-null    object
 7   Credit amount     954 non-null    int64 
 8   Duration          954 non-null    int64 
 9   Purpose           954 non-null    object
 10  Credit Risk       954 non-null    int64 
dtypes: int64(6), object(5)
memory usage: 82.1+ KB


None

Unique values of the categorical features:
	- Sex: ['male' 'female']
	- Housing: ['own' 'free' 'rent']
	- Saving accounts: [nan 'little' 'quite rich' 'rich' 'moderate']
	- Checking account: ['little' 'moderate' nan 'rich']
	- Purpose: ['radio/TV' 'education' 'furniture/equipment' 'car' 'business'
 'domestic appliances' 'repairs' 'vacation/others']


In [3]:
preprocessed_data = original_data.copy()

# For savings and checking accounts, we will replace the missing values with 'none':
preprocessed_data['Saving accounts'].fillna('none', inplace=True)
preprocessed_data['Checking account'].fillna('none', inplace=True)

# Dropping index column:
preprocessed_data.drop(columns=['Unnamed: 0'], inplace=True)

# Using pd.dummies to one-hot-encode the categorical features
preprocessed_data["Job"] = preprocessed_data["Job"].map({0: 'unskilled_nonresident', 1: 'unskilled_resident',
                                                         2: 'skilled', 3: 'highlyskilled'})

categorical_features = preprocessed_data.select_dtypes(include='object').columns
numerical_features = preprocessed_data.select_dtypes(include='number').columns.drop('Credit Risk')
print(f'Categorical features: {categorical_features}')
print(f'Numerical features: {numerical_features}')

preprocessed_data = pd.get_dummies(preprocessed_data, columns=categorical_features, dtype='int64')

# Remapping the target variable to 0 and 1:
preprocessed_data['Credit Risk'] = preprocessed_data['Credit Risk'].map({1: 0, 2: 1})

# Make sure all column names are valid python identifiers (important for pd.query() calls):
preprocessed_data.columns = preprocessed_data.columns.str.replace(' ', '_')
preprocessed_data.columns = preprocessed_data.columns.str.replace('/', '_')

# Normalizing the data
scaler = StandardScaler()
scaled_preprocessed_data = scaler.fit_transform(preprocessed_data)

display(preprocessed_data.head())
display(preprocessed_data.info())

display(scaled_preprocessed_data)

Categorical features: Index(['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account',
       'Purpose'],
      dtype='object')
Numerical features: Index(['Age', 'Credit amount', 'Duration'], dtype='object')


,Age,Credit_amount,Duration,Credit_Risk,Sex_female,Sex_male,Job_highlyskilled,Job_skilled,Job_unskilled_nonresident,Job_unskilled_resident,...,Checking_account_none,Checking_account_rich,Purpose_business,Purpose_car,Purpose_domestic_appliances,Purpose_education,Purpose_furniture_equipment,Purpose_radio_TV,Purpose_repairs,Purpose_vacation_others
0,67,1169,6,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
1,22,5951,48,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,49,2096,12,0,0,1,0,0,0,1,...,1,0,0,0,0,1,0,0,0,0
3,45,7882,42,0,0,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,53,4870,24,1,0,1,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954 entries, 0 to 953
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Age                          954 non-null    int64
 1   Credit_amount                954 non-null    int64
 2   Duration                     954 non-null    int64
 3   Credit_Risk                  954 non-null    int64
 4   Sex_female                   954 non-null    int64
 5   Sex_male                     954 non-null    int64
 6   Job_highlyskilled            954 non-null    int64
 7   Job_skilled                  954 non-null    int64
 8   Job_unskilled_nonresident    954 non-null    int64
 9   Job_unskilled_resident       954 non-null    int64
 10  Housing_free                 954 non-null    int64
 11  Housing_own                  954 non-null    int64
 12  Housing_rent                 954 non-null    int64
 13  Saving_accounts_little       954 non-null    int64

None

array([[ 2.7694545 , -0.7399179 , -1.22763429, ...,  1.62518349,
        -0.14633276, -0.11286653],
       [-1.18704073,  0.93690642,  2.26068929, ...,  1.62518349,
        -0.14633276, -0.11286653],
       [ 1.18685641, -0.41486224, -0.72930235, ..., -0.61531514,
        -0.14633276, -0.11286653],
       ...,
       [-1.0111965 , -0.39768023,  1.26402541, ..., -0.61531514,
        -0.14633276, -0.11286653],
       [-0.65950803,  0.29240557,  0.26736153, ..., -0.61531514,
        -0.14633276, -0.11286653],
       [-0.83535227,  2.69823821,  1.26402541, ..., -0.61531514,
        -0.14633276, -0.11286653]])

In [4]:
y = preprocessed_data['Credit_Risk']
X = preprocessed_data.drop(columns='Credit_Risk')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'ROC AUC: {roc_auc_score(y_test, y_pred)}')

Accuracy: 0.7696335078534031
ROC AUC: 0.6830357142857143


# Misc:

In [6]:
exp = LimeWrapper(clf, X_train, categorical_features)
evaluator = ExplanationModelEvaluator(clf, X_train, categorical_features, noise_gen_args={'encoding_dim': 5, 'epochs': 10})
evaluator.init()

Epoch 1/10


W0000 00:00:1733281179.004141  117317 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 1.2536 - val_loss: 1.2291
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2139 - val_loss: 1.2106
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1924 - val_loss: 1.1928
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1533 - val_loss: 1.1752
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1784 - val_loss: 1.1572
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1261 - val_loss: 1.1397
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1351 - val_loss: 1.1226
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1282 - val_loss: 1.1059
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1143 - val_loss: 1.0899
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1053 - val_loss: 1.0742
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step


In [10]:
evaluator.sensitivity(exp, X_test.iloc[0], iterations=100)

0.5406650246305418

In [11]:
evaluator._sensitivity_sequential(exp, X_test.iloc[0])

0.4381773399014778

# Apply the ensemble explainer

In [12]:
agg_explainer = AggregatedExplainer(explainer_types=[LimeWrapper, ShapTabularTreeWrapper, AnchorWrapper], 
                                    clf=clf, X_train=X_train, categorical_feature_names=categorical_features,
                                    noise_gen_args={'encoding_dim': 5, 'epochs': 10},
                                    evaluator_args={"debug": True})

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.2725 - val_loss: 1.2194
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.2472 - val_loss: 1.2008
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2159 - val_loss: 1.1831
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1975 - val_loss: 1.1655
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1721 - val_loss: 1.1483
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1573 - val_loss: 1.1320
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1396 - val_loss: 1.1160
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1349 - val_loss: 1.0999
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1116 - val_loss: 1.0842
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1075 - val_loss: 1.0693
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 681us/step


In [13]:
sample_idx = 0
agg_explanation = agg_explainer.explain_instance(X_test.iloc[sample_idx])

[faithfullness_correlation()]: Iteration 0
[faithfullness_correlation()]: Iteration 1
[faithfullness_correlation()]: Iteration 2
[faithfullness_correlation()]: Iteration 3
[faithfullness_correlation()]: Iteration 4
[faithfullness_correlation()]: Iteration 5
[faithfullness_correlation()]: Iteration 6
[faithfullness_correlation()]: Iteration 7
[faithfullness_correlation()]: Iteration 8
[faithfullness_correlation()]: Iteration 9
[faithfullness_correlation()]: Iteration 0
[faithfullness_correlation()]: Iteration 1
[faithfullness_correlation()]: Iteration 2
[faithfullness_correlation()]: Iteration 3
[faithfullness_correlation()]: Iteration 4
[faithfullness_correlation()]: Iteration 5
[faithfullness_correlation()]: Iteration 6
[faithfullness_correlation()]: Iteration 7
[faithfullness_correlation()]: Iteration 8
[faithfullness_correlation()]: Iteration 9
[faithfullness_correlation()]: Iteration 0
[faithfullness_correlation()]: Iteration 1
[faithfullness_correlation()]: Iteration 2
[faithfulln

In [8]:
display(agg_explanation)

,feature,score
0,Purpose_furniture_equipment,0.863380
1,Duration,0.804231
2,Age,0.773897
3,Checking_account_little,0.594576
4,Housing_own,0.453086
5,Credit_amount,0.323981
6,Saving_accounts_moderate,0.289085
7,Purpose_car,0.273221
8,Sex_female,0.272713
9,Housing_free,0.271590


In [12]:
agg_explainer.xai_evaluator._sensitivity_sequential(
    agg_explainer, 
    X_test.iloc[sample_idx],
    extra_explainer_params={
        "explainer_types": [LimeWrapper, ShapTabularTreeWrapper, AnchorWrapper],
        "noise_gen_args": {'encoding_dim': 5, 'epochs': 10},
        "evaluator": agg_explainer.xai_evaluator
    },
    iterations=1
)

[sensitivity()]: Iteration 0
[faithfullness_correlation()]: Iteration 0
[faithfullness_correlation()]: Iteration 1
[faithfullness_correlation()]: Iteration 2
[faithfullness_correlation()]: Iteration 3
[faithfullness_correlation()]: Iteration 4
[faithfullness_correlation()]: Iteration 5
[faithfullness_correlation()]: Iteration 6
[faithfullness_correlation()]: Iteration 7
[faithfullness_correlation()]: Iteration 8
[faithfullness_correlation()]: Iteration 9
[faithfullness_correlation()]: Iteration 0
[faithfullness_correlation()]: Iteration 1
[faithfullness_correlation()]: Iteration 2
[faithfullness_correlation()]: Iteration 3
[faithfullness_correlation()]: Iteration 4
[faithfullness_correlation()]: Iteration 5
[faithfullness_correlation()]: Iteration 6
[faithfullness_correlation()]: Iteration 7
[faithfullness_correlation()]: Iteration 8
[faithfullness_correlation()]: Iteration 9
[faithfullness_correlation()]: Iteration 0
[faithfullness_correlation()]: Iteration 1
[faithfullness_correlatio

0.9998768397064113

In [13]:
faithfulness = agg_explainer.xai_evaluator.faithfullness_correlation(agg_explainer, X_test.iloc[sample_idx])
sensitivity = agg_explainer.xai_evaluator._sensitivity_sequential(
                                                agg_explainer, X_test.iloc[sample_idx],
                                                extra_explainer_params={
                                                    "explainer_types": [LimeWrapper, ShapTabularTreeWrapper, AnchorWrapper],
                                                    "noise_gen_args": {'encoding_dim': 5, 'epochs': 10},
                                                    "evaluator": agg_explainer.xai_evaluator
                                                },
                                                iterations=1
                                            )
complexity = agg_explainer.xai_evaluator.complexity(agg_explainer, X_test.iloc[sample_idx])

metrics = agg_explainer.last_explanation_metrics.copy()
metrics.loc[AggregatedExplainer.__name__] = [faithfulness, sensitivity, complexity]

[faithfullness_correlation()]: Iteration 0
[faithfullness_correlation()]: Iteration 1
[faithfullness_correlation()]: Iteration 2
[faithfullness_correlation()]: Iteration 3
[faithfullness_correlation()]: Iteration 4
[faithfullness_correlation()]: Iteration 5
[faithfullness_correlation()]: Iteration 6
[faithfullness_correlation()]: Iteration 7
[faithfullness_correlation()]: Iteration 8
[faithfullness_correlation()]: Iteration 9
[faithfullness_correlation()]: Iteration 0
[faithfullness_correlation()]: Iteration 1
[faithfullness_correlation()]: Iteration 2
[faithfullness_correlation()]: Iteration 3
[faithfullness_correlation()]: Iteration 4
[faithfullness_correlation()]: Iteration 5
[faithfullness_correlation()]: Iteration 6
[faithfullness_correlation()]: Iteration 7
[faithfullness_correlation()]: Iteration 8
[faithfullness_correlation()]: Iteration 9
[faithfullness_correlation()]: Iteration 0
[faithfullness_correlation()]: Iteration 1
[faithfullness_correlation()]: Iteration 2
[faithfulln

In [17]:
metrics

,faithfullness_correlation,sensitivity,complexity
method,,,
LimeWrapper,0.374373,0.481724,2.570415
ShapTabularTreeWrapper,0.037084,0.718778,2.516309
AnchorWrapper,0.449357,0.876640,1.014089
AggregatedExplainer,0.595709,1.000000,3.183708


In [18]:
def display_ranking_df(ranking_df: pd.DataFrame, epsilon: float = 0.01):
    # If two features have a score difference smaller than epsilon, they are considered to have the same rank
    rank = pd.DataFrame(columns=['Feature', 'Rank'])
    rank['Feature'] = ranking_df["feature"]
    
    # Sort the ranking dataframe by score in descending order
    ranking_df = ranking_df.sort_values(by='score', ascending=False).reset_index(drop=True)
    
    current_rank = 1
    rank['Rank'] = 0
    rank.at[0, 'Rank'] = current_rank
    
    for i in range(1, len(ranking_df)):
        if abs(ranking_df.at[i, 'score'] - ranking_df.at[i-1, 'score']) < epsilon:
            rank.at[i, 'Rank'] = current_rank
        else:
            current_rank += 1
            rank.at[i, 'Rank'] = current_rank
    
    display(rank)


In [19]:
display_ranking_df(agg_explanation, epsilon=0.001)

,Feature,Rank
0,Purpose_furniture_equipment,1
1,Duration,2
2,Age,3
3,Checking_account_little,4
4,Housing_own,5
5,Credit_amount,6
6,Saving_accounts_moderate,7
7,Purpose_car,8
8,Sex_female,8
9,Housing_free,9
